In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
import pickle
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import seaborn as sns
import heapq
import bottleneck
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
%matplotlib inline

/home/lixinhui/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
mall_list = pd.read_csv('../others/mall_ID.csv',names=['mall_id'])
mall_list = list(mall_list['mall_id'])

f5 = open("../mydata/shop_label_dict.txt","rb")
shop_label_dict = pickle.load(f5)
f5.close()
f6 = open("../mydata/label_shop_dict.txt","rb")
label_shop_dict = pickle.load(f6)
f6.close()

f6 = open("../begain/wifi_count_dict.txt","rb")
wifi_count_dict = pickle.load(f6)
f6.close()
f5 = open("../begain/wifi_value_dict.txt","rb")
wifi_value_dict = pickle.load(f5)
f5.close()

df_train_data = pd.read_csv("../begain/df_train_data.csv")
df_vali_data= pd.read_csv("../begain/df_vali_data.csv")
df_test_data = pd.read_csv("../begain/df_test_data.csv")

In [9]:
vali_label_list = []
vali_predict_list = []
forest_acc_dict_baseline ={}
#mall_list2 = ['m_1263']#,'m_1175'
#mall_list2 = ['m_2415']#m_1175'
mall_list2 = ['m_1263']
index_strength = ['s_0','s_1','s_2','s_3','s_4','s_5','s_6','s_7','s_8','s_9']
index_bssid = ['b_0','b_1','b_2','b_3','b_4','b_5','b_6','b_7','b_8','b_9']
for mall_id_ in mall_list:
    
    test_pre_path2  = '../begain/data_wifi/test_'+ mall_id_ +'.csv'
    train_pre_path2 = '../begain/data_wifi/train_'+ mall_id_ +'.csv'
    vali_pre_path2 = '../begain/data_wifi/vali_'+ mall_id_ +'.csv'
    df_train_m = pd.read_csv(train_pre_path2,index_col ="row_id")
    df_vali_m = pd.read_csv(vali_pre_path2,index_col ="row_id")
    df_test_m = pd.read_csv(test_pre_path2,index_col ="row_id")
    
    save_path = '../data/1101/forest/test_'+ mall_id_ +'.csv'
    test_pre_path3  = '../begain/data_wifi_vector_important/test_'+ mall_id_ +'.csv'
    train_pre_path3 = '../begain/data_wifi_vector_important/train_'+ mall_id_ +'.csv'
    vali_pre_path3 = '../begain/data_wifi_vector_important/vali_'+ mall_id_ +'.csv'
   
    test_pre_path4  = '../begain/baseline_xgboost/test_'+ mall_id_ +'.csv'
    train_pre_path4 = '../begain/baseline_xgboost/train_'+ mall_id_ +'.csv'
    vali_pre_path4 = '../begain/baseline_xgboost/vali_'+ mall_id_ +'.csv'
    
    df_train_w = pd.read_csv(train_pre_path3,index_col ="row_id")
    df_vali_w = pd.read_csv(vali_pre_path3,index_col ="row_id")
    df_test_w = pd.read_csv(test_pre_path3,index_col ="row_id")

    df_train_xg = pd.read_csv(train_pre_path4,index_col ="row_id")
    df_vali_xg = pd.read_csv(vali_pre_path4,index_col ="row_id")
    df_test_xg = pd.read_csv(test_pre_path4,index_col ="row_id")
    

    df_train_m = df_train_m.join(df_train_w,how='left')
    df_vali_m = df_vali_m.join(df_vali_w,how='left')
    df_test_m = df_test_m.join(df_test_w,how='left')
    
#    df_train_m = df_train_m.join(df_train_xg,how='left')
#    df_vali_m = df_vali_m.join(df_vali_xg,how='left')
#    df_test_m = df_test_m.join(df_test_xg,how='left')
    
    train_data_row = df_train_m
    vali_data_row = df_vali_m
    test_data_row = df_test_m
    
    train_data_row['label']=train_data_row['shop_id']
#    train_drop_list = list(train_data_row.columns)[0:7]
    train_drop_list =[ 'user_id', 'shop_id', 'time_stamp', 'wifi_infos','mall_id']
    train_data_drop = train_data_row.drop(train_drop_list, axis=1)
    train_data_drop = train_data_drop.drop(index_bssid , axis=1)
    train_data_drop = train_data_drop.drop(index_strength, axis=1)
    train_data_drop['label']= train_data_drop['label'].apply(lambda x :shop_label_dict[mall_id_][x])
  
    vali_data_row['label']= vali_data_row['shop_id']
#    vali_drop_list = list(vali_data_row.columns)[0:7]

    vali_drop_list =[ 'user_id', 'shop_id', 'time_stamp', 'wifi_infos','mall_id']
    vali_data_drop = vali_data_row.drop(vali_drop_list, axis=1)
    vali_data_drop = vali_data_drop.drop(index_bssid , axis=1)
    vali_data_drop = vali_data_drop.drop(index_strength, axis=1)
    vali_data_drop['label']= vali_data_drop['label'].apply(lambda x :shop_label_dict[mall_id_][x])
    train_data_all = train_data_drop.append(vali_data_drop)
    
    test_drop_list =[ 'user_id', 'time_stamp', 'wifi_infos','mall_id']
    test_data_drop = test_data_row.drop(test_drop_list, axis=1)
    test_data_drop = test_data_drop.drop(index_bssid , axis=1)
    test_data_drop = test_data_drop.drop(index_strength, axis=1)
    
    
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_data_all['label'].values))
    train_data_drop['label'] = lbl.transform(list(train_data_drop['label'].values)) 
    vali_data_drop['label'] = lbl.transform(list(vali_data_drop['label'].values)) 
    train_data_all['label'] = lbl.transform(list(train_data_all['label'].values))
#    y_vali = vali_data_drop['label'].values
 #   train_data_drop = train_data_drop.append(vali_data_drop)
    
    np_train = np.array(train_data_all.fillna(value=0).values) 
    X_train = np_train[:,:-1] 
    y_train = np_train[:,-1].astype(int)
    
    
#    np_train = np.array(train_data_drop.fillna(value=0).values) 
#    X_train = np_train[:,:-1] 
#    y_train = np_train[:,-1].astype(int)
#    X_train = X_train[:,111:]
        
    np_vali = np.array(vali_data_drop.fillna(value=0).values) 
    X_vali = np_vali[:,:-1] 
    y_vali = np_vali[:,-1].astype(int)

#    X_vali = X_vali[:,111:]
    np_test = np.array(test_data_drop.fillna(value=0).values) 
    X_test = np_test
    
    
    clf = RandomForestClassifier(n_estimators =100, min_samples_split=3,max_features=40,random_state=10)
    clf.fit(X_train, y_train)
    
    tree_model = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_vali)
    predictions = [round(value) for value in y_pred]
    vali_label_list.append(y_vali)
    vali_predict_list.append(y_pred)
    accuracy = accuracy_score(y_vali, predictions)
    print mall_id_
    print("Accuracy: %.2f%%" % (accuracy * 100.0))
    forest_acc_dict_baseline[mall_id_] = accuracy
    test_pre = clf.predict(X_test)
    label_np = np.array(test_pre)
#    columns_list = list(np.sort(train_data_drop['label'].unique()))
    test_pre_df = pd.DataFrame(index = test_data_row.index, columns=['shop_id'],data = label_np)
    test_pre_df['shop_id'] = test_pre_df['shop_id'].apply(lambda x:lbl.inverse_transform(int(x)))
    test_pre_df.to_csv(save_path)
forest_baseline = pd.Series(forest_acc_dict_baseline)
forest_baseline.to_csv('../data/1101/forest/forest_baseline.csv')

m_1021
Accuracy: 100.00%
m_1085
Accuracy: 100.00%
m_1089
Accuracy: 100.00%
m_1175
Accuracy: 99.95%
m_1263
Accuracy: 99.70%
m_1293
Accuracy: 99.92%
m_1375
Accuracy: 100.00%
m_1377
Accuracy: 100.00%
m_1409
Accuracy: 100.00%
m_1621
Accuracy: 99.87%
m_1790
Accuracy: 100.00%
m_1831
Accuracy: 100.00%
m_1920
Accuracy: 100.00%
m_1950
Accuracy: 99.86%
m_2009
Accuracy: 100.00%
m_2058
Accuracy: 100.00%
m_2123
Accuracy: 100.00%
m_2182
Accuracy: 100.00%
m_2224
Accuracy: 99.76%
m_2267
Accuracy: 99.68%
m_2270
Accuracy: 100.00%
m_2333
Accuracy: 100.00%
m_2415
Accuracy: 100.00%
m_2467
Accuracy: 100.00%
m_2578
Accuracy: 100.00%
m_2715
Accuracy: 100.00%
m_2878
Accuracy: 100.00%
m_2907
Accuracy: 100.00%
m_3005
Accuracy: 99.92%
m_3019
Accuracy: 100.00%
m_3054
Accuracy: 100.00%
m_3112
Accuracy: 100.00%
m_3313
Accuracy: 100.00%
m_3425
Accuracy: 100.00%
m_3445
Accuracy: 100.00%
m_3501
Accuracy: 100.00%
m_3517
Accuracy: 100.00%
m_3528
Accuracy: 99.75%
m_3739
Accuracy: 100.00%
m_3832
Accuracy: 100.00%
m_3839
Ac

In [10]:
mall_0 = mall_list[0]
mall_other = mall_list[1:]
df_pre = pd.read_csv('../data/1101/forest/test_'+ mall_0 +'.csv',index_col=['row_id'])
df_pre['shop_id'] = df_pre['shop_id'].apply(lambda x :label_shop_dict[mall_0][int(x)])
df_sub = df_pre
for mall_id_ in mall_other:
    test_pre_path = '../data/1101/forest/test_'+ mall_id_ +'.csv'
    df_pre = pd.read_csv(test_pre_path,index_col=['row_id'])
    df_pre['shop_id'] = df_pre['shop_id'].apply(lambda x :label_shop_dict[mall_id_][int(x)])
    df_sub = df_sub.append(df_pre)
df_sub.to_csv('../submit/1101/result.csv')

In [11]:
df_sub.shape

(483931, 1)

In [12]:
df_sub.head()

,shop_id
row_id,
119090,s_783000
119168,s_235089
119243,s_396464
119667,s_3816514
119711,s_958323
